In [53]:
import requests
import pandas as pd
import threading
import json
import datetime
import time
import os
import numpy as np

def get_size(size):
    html_url = "https://staking-explorer2-268108.appspot.com/networks/harmony/validators_with_page?active=false&page=0&search=&size={}&sortOrder=desc&sortProperty=apr".format(size)
    res = requests.get(html_url)
    content = json.loads(res.content)
    return content['total']

def get_validator(page, size):
    html_url = "https://staking-explorer2-268108.appspot.com/networks/harmony/validators_with_page?active=false&page={}&search=&size={}&sortOrder=desc&sortProperty=apr".format(page, size)
    res = requests.get(html_url)
    content = json.loads(res.content)
    return content['validators']

def get_information(url, method, params) -> dict:
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    if r.status_code != 200:
        print("Error: Return status code %s" % r.status_code)
        return None
    try:
        content = json.loads(r.content)
    except ValueError:
        print("Error: Unable to read JSON reply")
        return None
    if "error" in content:
        print("Error: The method does not exist/is not available")
        return None
    else:
        return content['result']

def getAllValidatorInformation():
    url = 'https://api.s0.t.hmny.io/'
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(url, method, params)

# # get the total validator number
# curr_val = get_size(1)

# pages = curr_val // 100 + 1
# # get the validator info
# validator = []
# for i in range(pages):
#     val = get_validator(i,100)
#     validator.extend(val)

# df = pd.DataFrame(validator, columns = ['address', 'name','apr','total_stake', 'rate','uptime_percentage'])
# df['total_stake'] = df['total_stake'].apply(lambda c: int(float(c)/1e18))
# df['rate'] = df.apply(lambda c: "{0:.2f}%".format(float(c['rate']) * 100), axis = 1)
# df['apr'] = df.apply(lambda c: "{0:.2f}%".format(float(c['apr']) * 100), axis = 1)
# df['uptime_percentage'] = df['uptime_percentage'].apply(lambda c: None if np.isnan(c) else "{0:.2f}%".format(float(c) * 100))
# df.rename(columns={"rate": "fees", "apr": "expected_return", "uptime_percentage": "uptime"})


val_infos = getAllValidatorInformation()
address = []
name = []
apr = []
uptime = []
stake = []
rate = []
epos_status = []
boot_status = []
for i in val_infos:
    address.append(i['validator']['address'])
    name.append(i['validator']['name'])
    apr.append("{0:.2f}%".format(float(i['lifetime']['apr'])*100))
    sign_info = i['lifetime']['blocks']
    if float(sign_info['to-sign']) != 0:
        sign_perc = float(sign_info['signed'])/float(sign_info['to-sign'])
        uptime.append("{0:.2f}%".format(sign_perc*100))
    else:
        uptime.append(None)    
    stake.append(int(float(i['total-delegation'])/1e18))
    rate.append("{0:.2f}%".format(float(i['validator']['rate'])))
    epos_status.append(i['epos-status'])
    boot_status.append(i['booted-status'])
df = pd.DataFrame(list(zip(address, name, apr, stake, rate, uptime, epos_status, boot_status)), columns =['address', 'name', 'apr', 'stake', 'fees', 'uptime', 'epos-status','boot_status'])
df

,address,name,apr,stake,fees,uptime,epos-status,boot_status
0,one1hwe68yprkhp5sqq5u7sm9uqu8jxz87fd7ffex7,CONTABO-Nuremberg-1,52.72%,1806906,0.00%,89.77%,currently elected,None
1,one1nef0c8kgy30delcckf87jaegdp7s7ntrknvajj,Sesameseed,84.50%,23417539,1.00%,93.21%,currently elected,None
2,one1h9z66rjlrely889rkhlaxf7chk3pv2ksy46eq7,CONTABO-Nuremberg-2,0.00%,2040822,0.00%,99.76%,eligible to be elected next epoch,lost epos auction
3,one1kyyt7j29h4uhtnuhfar5wmngntx4gterrkd8q9,Total Harmony,45.22%,1638017,0.05%,75.95%,currently elected,None
4,one1kf42rl6yg2avkjsu34ch2jn8yjs64ycn4n9wdj,🔨 P-OPS Team Validator,86.31%,136908809,0.05%,99.50%,currently elected,None
...,...,...,...,...,...,...,...,...
142,one16ryldqjwdf3uytw9n8gcllhghsv0dggy4e4gva,LEMONed_ONE,0.00%,1520000,0.03%,None,eligible to be elected next epoch,not booted
143,one1dkyjjf8s94nvml9rvksexlt4a78lu9u63ul8lf,Game of thrONE,0.00%,10000,0.00%,None,eligible to be elected next epoch,not booted
144,one13333jp9cs7ezutn5wrc9hqr8nvaxr8h4x0cak7,ΛURORΛ,0.00%,10000,0.01%,None,eligible to be elected next epoch,not booted
145,one12249q4sxs6ewrluv0zar8jxv4pmvuu7m7qgh5a,HARMONY-NODE-A,0.00%,0,0.01%,None,not eligible to be elected next epoch,not booted


In [56]:
def getDelegationByDelegator(address):
    url = 'https://api.s0.t.hmny.io/'
    method = "hmy_getDelegationsByDelegator"
    params = [address]
    return get_information(url, method, params)

In [57]:
i = 'one1a8avzz3hcvhfrh2tp7rdprpvwt838y9um9c2q7'
getDelegationByDelegator(i)

[{'validator_address': 'one1nef0c8kgy30delcckf87jaegdp7s7ntrknvajj',
  'delegator_address': 'one1a8avzz3hcvhfrh2tp7rdprpvwt838y9um9c2q7',
  'amount': 10000000000000000000000,
  'reward': 0,
  'Undelegations': []}]